In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image


2022-03-14 13:34:53.461181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 13:34:53.461224: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
model = tf.keras.models.load_model('model')

In [10]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [13]:
img = Image.open('300.jpg')
img_resize = img.copy()

In [14]:
img_resized = img_resize.resize((900,900))

In [15]:
img = (np.expand_dims(img_resized,0))

In [16]:
predictions_single = probability_model.predict(img)

print(predictions_single)

[[8.6825818e-24 4.2689783e-14 1.0000000e+00]]


['dense_1']